# Recommending Films for Box Office Success!

![image](https://vip-go.premiumbeat.com/wp-content/uploads/2022/02/vr_2.jpg)

*Image by DOP Eben Bolter on the LED volume stage at Rebellion Film Studios in Oxford, UK.*

# Background

## Loading Tools and Data

Import our data science tools.

In [1]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format','{:.2f}'.format)

### IMDB - SQL Database

In [2]:
zip_path = 'zippedData/im.db.zip'
extract_path = 'zippedData/'

with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall(extract_path)

db_path = os.path.join(extract_path, 'im.db')

conn = sqlite3.connect(db_path)
pd.read_sql("""
    SELECT *
    FROM sqlite_master
    WHERE type = 'table';
""",conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [3]:
df_movie_basics = pd.read_sql("""
    SELECT *
    FROM movie_basics;
""",conn)

df_directors = pd.read_sql("""
    SELECT *
    FROM directors;
""",conn)

df_known_for = pd.read_sql("""
    SELECT *
    FROM known_for;
""",conn)

df_movie_ratings = pd.read_sql("""
    SELECT *
    FROM movie_ratings;
""",conn)

df_persons = pd.read_sql("""
    SELECT *
    FROM persons;
""",conn)

df_writers = pd.read_sql("""
    SELECT *
    FROM writers;
""",conn)

In [4]:
df_movie_basics

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.00,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.00,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.00,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.00,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.00,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.00,None


In [5]:
df_directors

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502
...,...,...
291169,tt8999974,nm10122357
291170,tt9001390,nm6711477
291171,tt9001494,nm10123242
291172,tt9001494,nm10123248


In [6]:
df_known_for

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
...,...,...
1638255,nm9990690,tt9090932
1638256,nm9990690,tt8737130
1638257,nm9991320,tt8734436
1638258,nm9991320,tt9615610


In [7]:
df_movie_ratings

,movie_id,averagerating,numvotes
0,tt10356526,8.30,31
1,tt10384606,8.90,559
2,tt1042974,6.40,20
3,tt1043726,4.20,50352
4,tt1060240,6.50,21
...,...,...,...
73851,tt9805820,8.10,25
73852,tt9844256,7.50,24
73853,tt9851050,4.70,14
73854,tt9886934,7.00,5


In [8]:
df_persons

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [9]:
df_writers

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087
...,...,...
255868,tt8999892,nm10122246
255869,tt8999974,nm10122357
255870,tt9001390,nm6711477
255871,tt9004986,nm4993825


### CSV Datasets

In [10]:
df_bom_movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df_rt_movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t')
df_rt_movie_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t', encoding='latin1')
df_tmdb_movies = pd.read_csv('zippedData/tmdb.movies.csv.gz')
df_tn_movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

In [11]:
df_bom_movie_gross

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.00,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.00,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.00,664300000,2010
3,Inception,WB,292600000.00,535700000,2010
4,Shrek Forever After,P/DW,238700000.00,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.00,NaN,2018
3383,Edward II (2018 re-release),FM,4800.00,NaN,2018
3384,El Pacto,Sony,2500.00,NaN,2018
3385,The Swan,Synergetic,2400.00,NaN,2018


In [12]:
df_rt_movie_info

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [13]:
df_rt_movie_reviews

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


In [14]:
df_tmdb_movies.vote_average.count()

26517

In [15]:
df_tn_movie_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


# Rotten Tomatoes Dataset

In [16]:
def custom_string_interpolation(series):
    ffill_series = series.fillna(method='ffill')
    bfill_series = series.fillna(method='bfill')
    combined_series = ffill_series.combine_first(bfill_series)
    return combined_series

df_rt_movie_info = df_rt_movie_info[['id', 'synopsis', #'rating', 
                                     'genre', 'director', 'writer', 'theater_date', # 'dvd_date', #'currency', 
                                     'box_office', #'runtime','studio'
                                    ]]
mask = df_rt_movie_info[df_rt_movie_info['box_office'].notnull()]
mask['revenue'] = mask['box_office'].str.replace(',','').astype(int)

df_rt_movie_info = mask
df_rt_movie_info['director'] = df_rt_movie_info['director'].fillna('unknown')
df_rt_movie_info['writer'] = df_rt_movie_info['writer'].fillna('unknown')
df_rt_movie_info['theater_date'] = custom_string_interpolation(df_rt_movie_info['theater_date'])
df_rt_movie_info['theater_date'] = df_rt_movie_info['theater_date'].str[-4:]
df_rt_movie_info

,id,synopsis,genre,director,writer,theater_date,box_office,revenue
1,3,"New York City, not-too-distant-future: Eric Pa...",Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012,"600,000",600000
6,10,Some cast and crew from NBC's highly acclaimed...,Comedy,Jake Kasdan,Mike White,2002,"41,032,915",41032915
7,13,"Stewart Kane, an Irishman living in the Austra...",Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,2006,"224,114",224114
8,14,"""Love Ranch"" is a bittersweet love story that ...",Drama,Taylor Hackford,Mark Jacobson,2010,"134,904",134904
15,22,Two-time Academy Award Winner Kevin Spacey giv...,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,2010,"1,039,869",1039869
...,...,...,...,...,...,...,...,...
1541,1980,A band of renegades on the run in outer space ...,Action and Adventure|Science Fiction and Fantasy,Joss Whedon,Joss Whedon,2005,"25,335,935",25335935
1542,1981,"Money, Fame and the Knowledge of English. In I...",Comedy|Drama,Gauri Shinde,Gauri Shinde,2012,"1,416,189",1416189
1545,1985,A woman who joins the undead against her will ...,Horror|Mystery and Suspense,Sebastian Gutierrez,Sebastian Gutierrez,2007,"59,371",59371
1546,1986,Aki Kaurismaki's The Man Without a Past opens ...,Art House and International|Comedy|Drama,unknown,unknown,2002,"794,306",794306


In [17]:
df_rt_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 340 entries, 1 to 1555
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            340 non-null    int64 
 1   synopsis      340 non-null    object
 2   genre         340 non-null    object
 3   director      340 non-null    object
 4   writer        340 non-null    object
 5   theater_date  340 non-null    object
 6   box_office    340 non-null    object
 7   revenue       340 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 23.9+ KB


In [18]:
df_rt_movie_reviews = df_rt_movie_reviews[['id', 'review', 'rating', #'fresh', 'critic', 'top_critic', 'publisher','date'
                                          ]]
df_rt_movie_reviews

,id,review,rating
0,3,A distinctly gallows take on contemporary fina...,3/5
1,3,It's an allegory in search of a meaning that n...,NaN
2,3,... life lived in a bubble in financial dealin...,NaN
3,3,Continuing along a line introduced in last yea...,NaN
4,3,... a perverse twist on neorealism...,NaN
...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN
54428,2000,NaN,1/5
54429,2000,NaN,2/5
54430,2000,NaN,2.5/5


In [19]:
df_rt_movie_reviews['rating'].unique()
# df_rt_movie_reviews[df_rt_movie_reviews['rating'].str.len(2)]
# mask = df_rt_movie_reviews[df_rt_movie_reviews['rating'].str.len() == 2]

# grade_to_score = {'A+': 10, 'A': 9, 'A-': 8, 'B+': 7, 'B': 6, 'B-': 5, 'C+': 4, 'C': 3, 'C-': 2, 'D+': 1.5, 'D': 1.4, 'D-': 1.3, 'F+': 1.2, 'F': 1.1, 'F-': 1}

# df_rt_movie_reviews['rating'] = df_rt_movie_reviews['rating'].map(grade_to_score)

array(['3/5', nan, 'C', '2/5', 'B-', '2/4', 'B', '3/4', '4/5', '4/4',
       '6/10', '1/4', '8', '2.5/4', '4/10', '2.0/5', '3/10', '7/10', 'A-',
       '5/5', 'F', '3.5/4', 'D+', '1.5/4', '3.5/5', '8/10', 'B+', '9/10',
       '2.5/5', '7.5/10', '5.5/10', 'C-', '1.5/5', '1/5', '5/10', 'C+',
       '0/5', '6', '0.5/4', 'D', '3.1/5', '3/6', '4.5/5', '0/4', '2/10',
       'D-', '7', '1/10', '3', 'A+', 'A', '4.0/4', '9.5/10', '2.5',
       '2.1/2', '6.5/10', '3.7/5', '8.4/10', '9', '1', '7.2/10', '2.2/5',
       '0.5/10', '5', '0', '2', '4.5', '7.7', '5.0/5', '8.5/10', '3.0/5',
       '0.5/5', '1.5/10', '3.0/4', '2.3/10', '4.5/10', '4/6', '3.5',
       '8.6/10', '6/8', '2.0/4', '2.7', '4.2/10', '5.8', '4', '7.1/10',
       '5/4', 'N', '3.5/10', '5.8/10', 'R', '4.0/5', '0/10', '5.0/10',
       '5.9/10', '2.4/5', '1.9/5', '4.9', '7.4/10', '1.5', '2.3/4',
       '8.8/10', '4.0/10', '2.2', '3.8/10', '6.8/10', '7.3', '7.0/10',
       '3.2', '4.2', '8.4', '5.5/5', '6.3/10', '7.6/10', '8.1/10',
  

In [20]:
df_rt_movie_reviews.info()
df_rt_movie_reviews

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      54432 non-null  int64 
 1   review  48869 non-null  object
 2   rating  40915 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


,id,review,rating
0,3,A distinctly gallows take on contemporary fina...,3/5
1,3,It's an allegory in search of a meaning that n...,NaN
2,3,... life lived in a bubble in financial dealin...,NaN
3,3,Continuing along a line introduced in last yea...,NaN
4,3,... a perverse twist on neorealism...,NaN
...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN
54428,2000,NaN,1/5
54429,2000,NaN,2/5
54430,2000,NaN,2.5/5


In [21]:
# df_rotten_tomatoes = pd.merge(df_rt_movie_

# df_rt_merged = pd.merge(df_rt_movie_info, df_rt_movie_reviews, on='id', how='outer')
# df_rt_merged
# df_rt_movie_info.info()

In [22]:
df_rt_movie_info.info()



<class 'pandas.core.frame.DataFrame'>
Index: 340 entries, 1 to 1555
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            340 non-null    int64 
 1   synopsis      340 non-null    object
 2   genre         340 non-null    object
 3   director      340 non-null    object
 4   writer        340 non-null    object
 5   theater_date  340 non-null    object
 6   box_office    340 non-null    object
 7   revenue       340 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 23.9+ KB


In [23]:
try:
    print('SUCCESS! All cells were executed without errors.')
except:
    print('FAILED! Error on indicated cell.')

SUCCESS! All cells were executed without errors.
